In [1]:
import os
from os.path import join as pj
import argparse
import sys
sys.path.append("modules")
import utils
import numpy as np
import torch as th
import scib
import scib.metrics as me
import anndata as ad
import scipy
import pandas as pd
import re
import itertools
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from scipy.stats import pearsonr
import copy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from operator import itemgetter
from collections import Counter
from tqdm import tqdm

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--task', type=str, default='atlas_tissues_cl')
parser.add_argument('--real_task', type=str, default='atlas_tissues')
parser.add_argument('--reference', type=str, default='atlas_new')
parser.add_argument('--experiment', type=str, default='c_3')
parser.add_argument('--real_experiment', type=str, default='offline')
parser.add_argument('--model', type=str, default='default')
parser.add_argument('--init_model', type=str, default='')
parser.add_argument('--init_model_real', type=str, default='sp_latest')
parser.add_argument('--method', type=str, default='midas_embed')
o, _ = parser.parse_known_args()  # for python interactive
# o = parser.parse_args()

In [3]:
# Load latent variables of reference data
cfg_task = re.sub("_atlas|_generalize|_transfer|_ref_.*", "", o.task)
data_config = utils.gen_data_config(cfg_task)
data_config_ref = utils.gen_data_config(o.reference)
data_config_ref["raw_data_dirs"] += data_config["raw_data_dirs"]
data_config_ref["raw_data_frags"] += data_config["raw_data_frags"]
data_config_ref["combs"] = data_config["combs"]
data_config_ref["comb_ratios"] = data_config["comb_ratios"]
data_config_ref["s_joint"] = data_config["s_joint"]
data_config = utils.load_toml("configs/data.toml")[cfg_task]
for k, v in data_config.items():
    vars(o)[k] = v
model_config = utils.load_toml("configs/model.toml")["default"]
if o.model != "default":
    model_config.update(utils.load_toml("configs/model.toml")[o.model])
for k, v in model_config.items():
    vars(o)[k] = v
o.s_joint, o.combs, *_ = utils.gen_all_batch_ids(o.s_joint, o.combs)
o.pred_dir = pj("result", o.task, o.experiment, o.model, "predict", o.init_model)
pred = utils.load_predicted(o, group_by="subset")

c = [v["z"]["joint"][:, :o.dim_c] for v in pred.values()]
subset_num = 34
c_ref = np.concatenate(c[:subset_num], axis=0)
c_orgs = np.concatenate(c[subset_num:], axis=0)
c_all = np.concatenate([c_orgs, c_ref], axis=0)

o_real = copy.deepcopy(o)
o_real.task = o.real_task

o_real.pred_dir = pj("result", o_real.task, o.real_experiment, o.model, "predict", o.init_model_real)
pred_real = utils.load_predicted(o_real, group_by="subset")

c_real = [v["z"]["joint"][:, :o_real.dim_c] for v in pred_real.values()]
# c_real = np.concatenate(c_real, axis=0)
c_ref_real = np.concatenate(c_real[:subset_num], axis=0)
c_orgs_real = np.concatenate(c_real[subset_num:], axis=0)
c_all_real = np.concatenate([c_orgs_real, c_ref_real], axis=0)
c_all_real = np.concatenate([c_orgs_real, c_ref_real], axis=0)
nbrs_real = NearestNeighbors(n_neighbors=15000, algorithm='ball_tree').fit(c_all_real)
distances_real, indices_real = nbrs_real.kneighbors(c_all_real)
indices_real


nbrs_all = NearestNeighbors(n_neighbors=15000, algorithm='ball_tree').fit(c_all)
distances_all, indices_all = nbrs_all.kneighbors(c_all)
indices_all

  0%|          | 0/40 [00:00<?, ?it/s]

Loading predicted variables ...
Loading subset 0: z, joint
Loading subset 1: z, joint
Loading subset 2: z, joint


  0%|          | 0/26 [00:00<?, ?it/s]

Loading subset 3: z, joint
Loading subset 4: z, joint
Loading subset 5: z, joint


  0%|          | 0/28 [00:00<?, ?it/s]

...

  0%|          | 0/17 [00:00<?, ?it/s]

Loading subset 34: z, joint
Loading subset 35: z, joint
Loading subset 36: z, joint


100%|██████████| 17/17 [00:00<00:00, 324.39it/s]


Converting to numpy ...
Converting subset 0: s, joint
Converting subset 0: z, joint
Converting subset 1: s, joint
Converting subset 1: z, joint
Converting subset 2: s, joint
Converting subset 2: z, joint
Converting subset 3: s, joint
Converting subset 3: z, joint
Converting subset 4: s, joint
Converting subset 4: z, joint
Converting subset 5: s, joint
Converting subset 5: z, joint
Converting subset 6: s, joint
Converting subset 6: z, joint
Converting subset 7: s, joint
Converting subset 7: z, joint
Converting subset 8: s, joint
Converting subset 8: z, joint
Converting subset 9: s, joint
Converting subset 9: z, joint
Converting subset 10: s, joint
Converting subset 10: z, joint
Converting subset 11: s, joint
Converting subset 11: z, joint
Converting subset 12: s, joint
Converting subset 12: z, joint
Converting subset 13: s, joint
Converting subset 13: z, joint
Converting subset 14: s, joint
Converting subset 14: z, joint
Converting subset 15: s, joint
Converting subset 15: z, joint
Conv

array([[     0,   2755, 156305, ...,  48745, 124476, 127633],
       [     1,   2231,   1436, ..., 216293,  98148,   3475],
       [     2,   2075,  16500, ...,  36751, 167059, 110644],
       ...,
       [273247, 272576, 272969, ..., 158523,  25479, 100087],
       [273248, 272000, 271372, ...,  81099, 144553,  58367],
       [273249, 272504, 273088, ...,  61377, 131481,  62091]])

In [ ]:
neigh_size = [5, 10, 100, 1000, 2500, 5000, 10000, 15000]
overlap_num_list = []
sum_list = []
overlap_percent = []
for j in tqdm(neigh_size):
    overlap_num = 0
    sum_num = 0
    for i in range(len(c_orgs)):
        overlap_list = list(set(indices_all[i,1:j]) & set(indices_real[i,1:j]))
        overlap_num += len(overlap_list)
        sum_num += j-1
    overlap_num_list.append(overlap_num)
    sum_list.append(sum_num)
    overlap_percent.append(overlap_num/sum_num)

In [ ]:
neigh_size = [5, 10, 100, 1000, 2500, 5000, 10000, 15000]
overlap_num_list = []
sum_list_all = []
overlap_percent_all = []
for j in tqdm(neigh_size):
    overlap_num = 0
    sum_num = 0
    for i in range(len(c_all)):
        overlap_list = list(set(indices_all[i,1:j]) & set(indices_real[i,1:j]))
        overlap_num += len(overlap_list)
        sum_num += j-1
    overlap_num_list.append(overlap_num)
    sum_list_all.append(sum_num)
    overlap_percent_all.append(overlap_num/sum_num)

In [ ]:
fig = plt.figure()
ax=plt.axes()
ax.spines['bottom'].set_linewidth('1.0')
ax.spines['left'].set_linewidth('1.0')
ax.spines['top'].set_linewidth('1.0')
ax.spines['right'].set_linewidth('1.0')
plt.tick_params(axis="both", which="major", width=1, length=3)
plt.plot(neigh_size[:8], overlap_percent[:8], label='Query Neighborhood overlap', linewidth =1.5)
plt.plot(neigh_size[:8], overlap_percent_all[:8], label='Overall Neighborhood overlap', linewidth =1.5)

plt.xlabel('Neighborhood size', fontsize=12)
plt.ylabel('Neighborhood overlap', fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title('atlas_tissues_cl', fontsize=12)
plt.legend(prop = {'size':8}, frameon=False)
plt.savefig("overlap_tissues_cl.png")
plt.show()


In [ ]:
df = pd.DataFrame(overlap_percent)
df1 = pd.DataFrame(overlap_percent_all)
result_dir = pj("analysis", "overlap")
utils.mkdirs(result_dir, remove_old=False)
df.to_csv(pj(result_dir, "overlap_percent_tissues.csv"), index=False)
df1.to_csv(pj(result_dir, "overlap_percent_all_tissues.csv"), index=False)